### Import external python files

In [1]:
%matplotlib inline
%run ../skeleton.py
%run ../animation.py
%run ../realign_matrix.py

### Load the skeleton definition
This is pickled from maya

In [2]:
animations = [load_animation(r"../motion_graph/anim_{}.dat".format(i)) for i in range(28)]

### Draw Skeleton
We will use k3d to display the skeleton in a view port

In [3]:
import k3d

def plot_skeleton(plot, skeleton, width=0.05, color=0x2233FF):
    for i in range(len(skeleton._bones)):
        if skeleton._bones[i]._parentId >= 0:
            startM = skeleton.globalMatrix(i)
            endM = skeleton.globalMatrix(skeleton._bones[i]._parentId)
            p = k3d.line([startM[3][:3], endM[3][:3]], width=width, color=color)
            plot += p

def plot_anchor(plot, skeleton):
    p = k3d.points([
            skeleton.anchorGlobalPosition(i) for i in range(len(skeleton._anchors))
        ], point_size=0.02, color=0x55FF00)
    plot += p
    
def plot_animation(plot, skeleton, animation):
    #plot skeleton at 2 frames
    keycount, tracks = animation
    skeleton.load_animation(animation, int(keycount/3) )
    plot_skeleton(plot, skeleton)
    skeleton.load_animation(animation, int(keycount/3*2) )
    plot_skeleton(plot, skeleton)

    
    #plot trajectories
    boneIds = [skeleton.bone_id(n) for n in ['Hips','LeftHand','RightHand','LeftFoot','RightFoot']]
    lines = [[] for i in range(len(boneIds))]
    for frame in range(keycount):
        skeleton.load_animation(animation, frame )
        for i,boneId in enumerate(boneIds):
            lines[i].append(skeleton.globalMatrix(boneId)[3][:3])
            
    for i in range(len(boneIds)):
        p = k3d.line(lines[i], shader='simple', color=0xFF22FF)
        plot += p

In [9]:
import numpy as np
from scipy.spatial import Delaunay

def build_skeleton_edges(skeleton):
    edges = []
    extra_edges = []
    for bone in skeleton._bones[1:]:
        children = skeleton.bone_children(bone._id)
        #first connect all my children to me
        for b in children:
            edges.append([bone._id-1, b._id-1])
        #connect all the children together
        for a in children:
            for b in children:
                if a != b:
                    extra_edges.append([a._id-1, b._id-1])
    return edges, extra_edges

def build_point_list_from_skeleton(skeleton):
    points = [skeleton.globalMatrix(i)[3][:3] for i in range(1, len(skeleton._bones))]
    points += [skeleton.anchorGlobalPosition(i) for i in range(len(skeleton._anchors))]
    return np.array(points)

def triangulate_skeleton(skeleton):
    points = build_point_list_from_skeleton(skeleton)
    tri = Delaunay(points, qhull_options = 'Qt Qbb Qc').simplices
    
    #discard flat tetrahedron
    keep = np.ones(len(tri), dtype = bool)
    for i, t in enumerate(tri):
        if abs(np.linalg.det(np.hstack((points[t], np.ones([1,3+1]).T)))) < 1E-15:
            keep[i] = False # Point is coplanar, we don't want to keep it
    tri = tri[keep]
    
    #keep unique edges only
    edges = np.array([
        flattentup for tup in 
        [[[t[0],t[1]],[t[2],t[0]],[t[1],t[3]],[t[0],t[2]],[t[3],t[1]],[t[2],t[3]]] for t in tri] 
        for flattentup in tup])
    edges.sort()
    edges = np.unique(edges, axis=0)
    
    
    return points, edges
    
def plot_edges(plot, points, edges, color=0xFF3300):
    for edge in edges:
        p = k3d.line([points[edge[0]], points[edge[1]]], color=color)
        plot += p
            

In [8]:
from scipy import sparse

ANCHOR_WEIGHT = 1.0

def laplacianUmbrellaMatrix(points, edges, anchorsVertIds=None):
    """compute a laplacian matrix with umbrella weights"""
    anchorsVertIds = anchorsVertIds or []
    n = len(points)
    k = len(anchorsVertIds)
    I = []
    J = []
    V = []
    
    # Build sparse Laplacian Matrix coordinates and values
    for i in range(n):
        indices = [edge[0] for edge in edges if edge[1] == i]
        indices += [edge[1] for edge in edges if edge[0] == i]
        z = len(indices)
        I = I + ([i] * (z + 1)) # repeated row
        J = J + indices + [i] # column indices and this row
        V = V + ([-1] * z) + [z] # negative weights and row degree
        
    # augment Laplacian matrix with anchor weights  
    for i in range(k):
        I = I + [n + i]
        J = J + [anchorsVertIds[i]]
        V = V + [ANCHOR_WEIGHT] # default anchor weight
    
    L = sparse.coo_matrix((V, (I, J)), shape=(n + k, n)).tocsr()
    
    return L

def laplacianInvDistanceMatrix(points, edges, anchorsVertIds=None):
    """compute a laplacian matrix with umbrella weights"""
    anchorsVertIds = anchorsVertIds or []
    n = len(points)
    k = len(anchorsVertIds)
    I = []
    J = []
    V = []
    
    # Build sparse Laplacian Matrix coordinates and values
    for i in range(n):
        indices = [edge[0] for edge in edges if edge[1] == i]
        distances = [points[edge[0]]-points[edge[1]] for edge in edges if edge[1] == i]
        indices += [edge[1] for edge in edges if edge[0] == i]
        distances += [points[edge[0]]-points[edge[1]] for edge in edges if edge[0] == i]
        if len(indices)==0:
            continue
        if len(indices) == 1:
            weights = np.array([1.0])
        else:
            distances = np.array(distances)
            distances = np.sum(distances*distances, axis=1)
            weights = distances/np.max(distances)* -1
        
        
        z = len(indices)
        I = I + ([i] * (z + 1)) # repeated row
        J = J + indices + [i] # column indices and this row
        V = V + weights.tolist() + [-np.sum(weights)] # negative weights and row degree
        
    # augment Laplacian matrix with anchor weights  
    for i in range(k):
        I = I + [n + i]
        J = J + [anchorsVertIds[i]]
        V = V + [ANCHOR_WEIGHT] # default anchor weight
    
    L = sparse.coo_matrix((V, (I, J)), shape=(n + k, n)).tocsr()
    
    return L

In [ ]:
def test_compare_skeletons():
    human_skeleton = load_skeleton(r'../skeleton_retarget_human.dat')
    rhino_skeleton = load_skeleton(r'../skeleton_retarget_rhino.dat')
    
    human_anchor = [human_skeleton.anchorGlobalPosition(i) for i in range(len(human_skeleton._anchors))]
    rhino_anchor = [rhino_skeleton.anchorGlobalPosition(i) for i in range(len(rhino_skeleton._anchors))]

    plot = k3d.plot()
    plot_skeleton(plot, human_skeleton, color=0xff0000)   
    plot_skeleton(plot, rhino_skeleton, color=0x0000ff)
    plot_anchor(plot, human_skeleton)
    plot_anchor(plot, rhino_skeleton)
    
    for h,r in zip(human_anchor, rhino_anchor):
        p = k3d.line([h,r], color=0x005500)
        plot += p
    
    plot.display()
    
test_compare_skeletons()

In [ ]:
def test_triangulate():
    human_skeleton = load_skeleton(r'../skeleton_retarget_human.dat')
    human_skeleton.load_animation(animations[0], 10 )
    
    points, edges = triangulate_skeleton(human_skeleton)
    
    plot = k3d.plot()
    plot_skeleton(plot, human_skeleton, color=0xff0000)   
    plot_edges(plot, points, edges, 0x0033FF)
    plot.display()
    
test_triangulate()

In [78]:
def build_realigns_data (skel):
    realigns = []
    realignsIdx = []
    for bone in skel._bones[1:]:
        children = skel.bone_children(bone._id)
        if len(children)>0:
            world = skel.globalMatrix(bone._id)
            realigns.append(RealignMatrix(world, np.array([skel.globalMatrix(c._id)[3][:3]-world[3][:3] for c in children])))
            realignsIdx.append([c._id-1 for c in children])
        else:
            realigns.append(None)
            realignsIdx.append([])
    return realigns, realignsIdx
            
def realign_skeleton(skel, points, realigns, realignsIdx):
    skel._bones[1]._matrix[3][:3] = points[0]
    for bone, realign, realignidx in zip(skel._bones[1:], realigns, realignsIdx):
        if realign:
            worldM = skel.globalMatrix(bone._id)
            newWorldM = realign.solve_from_local_vectors(worldM, [points[idx]-worldM[3][:3] for idx in realignidx])
            skel.setGlobalMatrix(bone._id, newWorldM)
            
            
def build_local_realigns_data(skel):
    realignsIdx = []
    for bone in skel._bones[1:]:
        children = skel.bone_children(bone._id)
        if len(children)>0:
            realignsIdx.append(children[0]._id-1)
        else:
            realignsIdx.append(-1)
    return realignsIdx

def realign_local_skeleton(skel, points, realignsIdx):
    
    def get_M_aligned_to_x(M, x):
        x = x/np.linalg.norm(x)
        z = np.cross(x, M[1][:3])
        z = z/np.linalg.norm(z)
        y = np.cross(z, x)
        y = y/np.linalg.norm(y)
        return np.array([x.tolist()+[0],y.tolist()+[0],z.tolist()+[0],M[3]], dtype=float)
    
    skel._bones[1]._matrix[3][:3] = points[0]
    for bone, idx in zip(skel._bones[1:], realignsIdx):
        if idx > -1:
            world = skel.globalMatrix(bone._id)
            pt = points[idx]-world[3][:3]
            matrix = get_M_aligned_to_x(world, pt)
            
            local = np.dot(matrix, np.linalg.inv(skel.globalMatrix(bone._parentId)))
            local[3][:3] = bone._matrix[3][:3]
            bone._matrix = local


In [7]:
from scipy.sparse.linalg import lsqr

def solveLaplacianMesh(L, points, anchors, iterations=25):
    n = len(points)
    k = len(anchors)
    updated_vertices = points.copy()
    
    for _ in range (iterations):

        delta = np.array(L.dot(updated_vertices))

        # augment delta solution matrix with weighted anchors
        for i in range(k):
            delta[n + i, :] = ANCHOR_WEIGHT * anchors[i, :]

        # update mesh vertices with least-squares solution
        for i in range(3):
            updated_vertices[:, i] = lsqr(L, delta[:, i])[0]
    
    return updated_vertices

In [ ]:
from scipy.optimize import minimize

def test_solving_laplacian():
    human_skeleton = load_skeleton(r'../skeleton_retarget_human.dat')
    rhino_skeleton = load_skeleton(r'../skeleton_retarget_rhino.dat')
    
    realigns, realignsIdx = build_realigns_data(rhino_skeleton)
    
    human_animation = animations[0]
    local_animation = human_skeleton.convert_animation_to_local(human_animation)
    rhino_animation = rhino_skeleton.convert_back_local_animation(local_animation)

    human_skeleton.load_animation(human_animation, 10 )
    rhino_skeleton.load_animation(rhino_animation, 10 )
    
    points, edges = triangulate_skeleton(human_skeleton)
    bcount = len(human_skeleton._bones[1:])
    locked = list(range(bcount, len(points)))
    feetbone = [bone._id-1 for bone in human_skeleton._bones if bone._name in 
               ['Hips',
                'LeftUpLeg','LeftLeg','LeftFoot','LeftFootToes','LeftFootToesEnd',
                'RightUpLeg','RightLeg','RightFoot','RightFootToes','RightFootToesEnd']]
    extrabone = [bone._id-1 for bone in human_skeleton._bones if bone._name in 
               ['Spine','Spine1','Spine2','Neck','Neck1','Head',
                'LeftClavicle','RightClavicle','LeftArm','RightArm'
                ]]

    L = laplacianInvDistanceMatrix(points, edges)
    v0 = build_point_list_from_skeleton(rhino_skeleton)
    Lstart = L.dot(points)
    vcount = len(points)
    
    def _objective(flattenV):
        #lapacian energy minization
        V = flattenV.reshape((vcount,3))
        lv = L.dot(V) - Lstart
        laplace = np.sum(lv*lv) * 0.5
        
        #locked
        lv = (v0 - V)[locked]
        lock = np.sum(lv*lv) * 0.5
        
        lv = (v0 - V)[feetbone]
        feets = np.sum(lv*lv) * 0.5
        
        lv = (v0 - V)[extrabone]
        extras = np.sum(lv*lv) * 0.5
        
        return laplace + lock*10 + extras*10
    
    def _constraint(flattenV):
        V = flattenV.reshape((vcount,3))
        lv = (v0 - V)[feetbone+extrabone]
        return np.sum(lv*lv) * 0.5
        
    constraints = []
    constraints += [
        {'type':'eq', 'fun':_constraint}
    ]
        
    
    plot = k3d.plot()
    plot_skeleton(plot, rhino_skeleton, color=0xff0000)
    plot.display()
    
    v1 = points.copy()
    for i in range(15):
        sol = minimize(_objective, v1, method='SLSQP', constraints=constraints)
        v1 = sol.x.reshape((vcount,3))
        realign_skeleton(rhino_skeleton, v1, realigns, realignsIdx)
        #v1 = build_point_list_from_skeleton(rhino_skeleton)
        print (sol.message)
        if sol.success:
            break
        
    plot_skeleton(plot, rhino_skeleton, color=0x0000ff) 
    plot += k3d.points(v1[:bcount], point_size=0.02, color=0x55FF00)
    #plot_edges(plot, v1, edges, 0x0033FF)
    
test_solving_laplacian()

In [ ]:
from scipy.optimize import minimize

def test_solving_laplacian2():
    human_skeleton = load_skeleton(r'../skeleton_retarget_human.dat')
    rhino_skeleton = load_skeleton(r'../skeleton_retarget_rhino.dat')
    
    realigns, realignsIdx = build_realigns_data(rhino_skeleton)
    
    human_animation = animations[0]
    local_animation = human_skeleton.convert_animation_to_local(human_animation)
    rhino_animation = rhino_skeleton.convert_back_local_animation(local_animation)

    human_skeleton.load_animation(human_animation, 20 )
    rhino_skeleton.load_animation(rhino_animation, 20 )
    
    points, edges = triangulate_skeleton(human_skeleton)
    bcount = len(human_skeleton._bones[1:])
    locked = list(range(bcount, len(points)))
    bonesToMove = [bone._id-1 for bone in human_skeleton._bones if bone._name not in 
               [
                'LeftUpLeg','LeftLeg','LeftFoot','LeftFootToes','LeftFootToesEnd',
                'RightUpLeg','RightLeg','RightFoot','RightFootToes','RightFootToesEnd']]

    L = laplacianInvDistanceMatrix(points, edges)
    v0 = build_point_list_from_skeleton(rhino_skeleton)
    Lstart = L.dot(points)
    vcount = len(bonesToMove)
    
    def _objective(flattenV):
        #lapacian energy minization
        V = flattenV.reshape((vcount,3))
        v0[bonesToMove] = V
        lv = L.dot(v0) - Lstart
        laplace = np.sum(lv*lv) * 0.5
        return laplace 
    
    plot = k3d.plot()
    plot_skeleton(plot, rhino_skeleton, color=0xff0000)
    plot.display()
    
    v1 = v0[bonesToMove]
    for i in range(15):
        sol = minimize(_objective, v1, method='SLSQP')
        v1 = sol.x.reshape((vcount,3))
        print (sol.message)
        if sol.success:
            break
        
    v0[bonesToMove] = v1
    realign_skeleton(rhino_skeleton, v0, realigns, realignsIdx)
    
    plot_skeleton(plot, rhino_skeleton, color=0x0000ff) 
    plot += k3d.points(v0[bcount:], point_size=0.02, color=0x55FF00)
    #plot_edges(plot, v1, edges, 0x0033FF)
    
test_solving_laplacian2()

In [10]:
from scipy.optimize import minimize

def test_solving_laplacian_animation():
    human_skeleton = load_skeleton(r'../skeleton_retarget_human.dat')
    rhinoAligned_skeleton = load_skeleton(r'../skeleton_retarget_rhinoAligned.dat')
    rhino_skeleton = load_skeleton(r'../skeleton_retarget_rhino.dat')
    
    bcount = len(human_skeleton._bones[1:])
    bonesToMove = [bone._id-1 for bone in human_skeleton._bones if bone._name not in 
               ['Hips', 'Spine', 'Spine1', 'Spine2', 'Neck', 'Neck1', 'Head', 'HeadEnd',
                'LeftUpLeg','LeftLeg','LeftFoot','LeftFootToes','LeftFootToesEnd',
                'RightUpLeg','RightLeg','RightFoot','RightFootToes','RightFootToesEnd']]
    vcount = len(bonesToMove)
    
    realignsIdx = build_local_realigns_data(rhinoAligned_skeleton)
    
    human_animation = animations[0]
    local_animation = human_skeleton.convert_animation_to_local(human_animation)
    rhino_animation = rhinoAligned_skeleton.convert_back_local_animation(local_animation)
    
    framecount = human_animation[0]
    
    all_points = []
    all_edges = []
    all_L = []
    for frame in range(framecount):
        human_skeleton.load_animation(human_animation, frame )
        points, edges = triangulate_skeleton(human_skeleton)
        L = laplacianInvDistanceMatrix(points, edges)
        
        all_points.append(points)
        all_edges.append(edges)
        all_L.append(L)
    
    for iteration in range(5):
        print(iteration)
        rhinoAligned_skeleton = load_skeleton(r'../skeleton_retarget_rhinoAligned.dat')
        
        all_v0 = []
        all_v1 = []
        
        for frame in range(framecount):
            L = all_L[frame]
            points = all_points[frame]
            
            rhinoAligned_skeleton.load_animation(rhino_animation, frame )
            v0 = build_point_list_from_skeleton(rhinoAligned_skeleton)
            v0Ref = v0.copy()
            Lstart = L.dot(points)

            def _objective(flattenV):
                #lapacian energy minization
                V = flattenV.reshape((vcount,3))
                v0Ref[bonesToMove] = V
                lv = L.dot(v0Ref) - Lstart
                laplace = np.sum(lv*lv) * 0.5
                return laplace 

            v1 = v0[bonesToMove]
            for i in range(3):
                sol = minimize(_objective, v1, method='SLSQP')
                v1 = sol.x.reshape((vcount,3))
                if sol.success:
                    break

            all_v0.append(v0)
            all_v1.append(v1)
            
        smooth_v1 = [all_v1[0]]
        for frame in range(1,framecount-1):
            smooth_v1.append( (all_v1[frame-1] + all_v1[frame]*2 + all_v1[frame+1])*0.25 )
        smooth_v1 += [all_v1[-1]]
            
        tracks_buffer = rhinoAligned_skeleton.create_tracks_buffer()
        
        for frame in range(framecount):
            v0 = all_v0[frame]
            v1 = smooth_v1[frame]
            
            v0[bonesToMove] = v1*0.1 + v0[bonesToMove]*0.9
            realign_local_skeleton(rhinoAligned_skeleton, v0, realignsIdx)
            rhinoAligned_skeleton.save_tracks(tracks_buffer)
            

        rhino_animation = (framecount, tracks_buffer)
    
    
    rhinoAligned_skeleton = load_skeleton(r'../skeleton_retarget_rhinoAligned.dat')
    local_animation = rhinoAligned_skeleton.convert_animation_to_local(rhino_animation)
    rhino_animation = rhino_skeleton.convert_back_local_animation(local_animation)
    
    return rhino_animation
    
    
    
retargeted_animation = test_solving_laplacian_animation()

display_skeleton = load_skeleton(r'../skeleton_retarget_rhino.dat')
plot = k3d.plot()
plot_animation(plot, display_skeleton, retargeted_animation)   
plot.display()
save_animation(r'retargeted_rhino_animation.dat', retargeted_animation[1])

NameError: name 'build_local_realigns_data' is not defined